<a href="https://colab.research.google.com/github/Iramuk-ganh/rl/blob/main/practice_vi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Algorithm
-initialize state_values as zeros

-find q_value function q(s,a) i.e reward due to action a for first step and then following greedy policy wrt value function

-find new_state_values as max q(s,a) over all possible actions

-max_diff = max(new_state_values - state_values)

-state_values = new_state_values

-repeat until max_diff < 0.01

###Code for display

In [ ]:
import sys, os
if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash

    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/coursera/grading.py -O ../grading.py
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/coursera/week2_model_based/submit.py
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/coursera/week2_model_based/mdp.py

    !touch .setup_complete

# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

Selecting previously unselected package xvfb.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.10_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.10) ...
Setting up xvfb (2:1.19.6-1ubuntu4.10) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Starting virtual X frame buffer: Xvfb.


###Three Levels of nested dictionary is used in transition_probs

In [ ]:
transition_probs = {
    's0':{
       'a0':{'s0':0.5, 's2':0.5},
       'a1':{'s2':1}
    },
    's1': {
        'a0': {'s0': 0.7, 's1': 0.1, 's2': 0.2},
        'a1': {'s1': 0.95, 's2': 0.05}
    },
    's2': {
        'a0': {'s0': 0.4, 's2': 0.6},
        'a1': {'s0': 0.3, 's1': 0.3, 's2': 0.4}
    }
}

rewards ={
    's1':{'a0':{'s0': +5}},
    's2':{'a1':{'s0':-1}}
}

In [ ]:
from mdp import MDP

In [ ]:
mdp = MDP(transition_probs, rewards, initial_state='s0')

In [ ]:
print('initial state =', mdp.reset())
next_state, reward, done, info = mdp.step('a1')
print('next_state = %s, reward = %s, done = %s' % (next_state, reward, done))

initial state = s0
next_state = s2, reward = 0.0, done = False


In [ ]:
print("mdp.get_all_states =", mdp.get_all_states())
print("mdp.get_possible_actions('s1') = ", mdp.get_possible_actions('s1'))
print("mdp.get_next_states('s1', 'a0') = ", mdp.get_next_states('s1', 'a0'))
print("mdp.get_reward('s1', 'a0', 's0') = ", mdp.get_reward('s1', 'a0', 's0'))
print("mdp.get_transition_prob('s1', 'a0', 's0') = ", mdp.get_transition_prob('s1', 'a0', 's0'))

mdp.get_all_states = ('s0', 's1', 's2')
mdp.get_possible_actions('s1') =  ('a0', 'a1')
mdp.get_next_states('s1', 'a0') =  {'s0': 0.7, 's1': 0.1, 's2': 0.2}
mdp.get_reward('s1', 'a0', 's0') =  5
mdp.get_transition_prob('s1', 'a0', 's0') =  0.7


### Value Iteration

Now let's build something to solve this MDP. The simplest algorithm so far is __V__alue __I__teration

Here's the pseudo-code for VI:

---

`1.` Initialize $V^{(0)}(s)=0$, for all $s$

`2.` For $i=0, 1, 2, \dots$
 
`3.` $ \quad V_{(i+1)}(s) = \max_a \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')]$, for all $s$

---

First, let's write a function to compute the state-action value function $Q^{\pi}$, defined as follows

$$Q_i(s, a) = \sum_{s'} P(s' | s,a) \cdot [ r(s,a,s') + \gamma V_{i}(s')]$$


In [ ]:
import numpy as np

In [ ]:
def get_action_value(mdp, state_values, state, action, gamma):
    """ Computes Q(s,a) as in formula above """

    possible_states = mdp.get_next_states(state, action)
    l = [mdp.get_transition_prob(state, action, s)*(mdp.get_reward(state, action, s) + gamma * state_values[s]) for s in possible_states]
    q_value = np.sum(l)

    return q_value

In [ ]:
test_Vs = {s: i for i, s in enumerate(sorted(mdp.get_all_states()))}
assert np.isclose(get_action_value(mdp, test_Vs, 's2', 'a1', 0.9), 0.69)
assert np.isclose(get_action_value(mdp, test_Vs, 's1', 'a0', 0.9), 3.95)

###np.sum(list) can be done for a list too.

In [ ]:
l = [1, 2, 3]
np.sum(l)

6

###Making dictionary using enumerate and iterable objects like array, list

In [ ]:
a = np.arange(4, 16, 4)
dict = {s:i for i,s in enumerate(a)}
a, dict

(array([ 4,  8, 12]), {4: 0, 8: 1, 12: 2})

In [ ]:
def get_new_state_value(mdp, state_values, state, gamma):
    """ Computes next V(s) as in formula above. Please do not change state_values in process. """
    if mdp.is_terminal(state):
        return 0

    possible_actions = mdp.get_possible_actions(state)
    value = np.max([get_action_value(mdp, state_values, state, a, gamma) for a in possible_actions])
    
    return value

In [ ]:
test_Vs_copy = test_Vs
assert np.isclose(get_new_state_value(mdp, test_Vs, 's0', 0.9), 1.8)
assert np.isclose(get_new_state_value(mdp, test_Vs, 's2', 0.9), 1.08)
assert np.isclose(get_new_state_value(mdp, {'s0': -1e10, 's1': 0, 's2': -2e10}, 's0', 0.9), -13500000000.0), \
    "Please ensure that you handle negative Q-values of arbitrary magnitude correctly"
assert test_Vs == test_Vs_copy, "Please do not change state_values in get_new_state_value"

###Finally, let's combine everything we wrote into a working value iteration algorithm.

In [ ]:
gamma = 0.9#close to 1, near-sighted
num_iter = 100

#stopping condition
min_difference = 0.001

#initialize V(s)
state_values = {s:0 for s in mdp.get_all_states()}

for i in range(num_iter):
  # Compute new state values using the functions you defined above.
  # It must be a dict {state : float V_new(state)}
  new_state_values = {s:get_new_state_value(mdp, state_values, s, gamma) for s in mdp.get_all_states()}

  # Compute difference
  diff = max(abs(new_state_values[s] - state_values[s]) for s in mdp.get_all_states())
  print("iter%4i  |   diff:%6.5f   |   " %(i, diff), end="") 
  print('   '.join("V(%s) = %.3f" % (s, v) for s, v in state_values.items()))
  state_values = new_state_values

  if diff < min_difference:
      print("Terminated")
      break

iter   0  |   diff:3.50000   |   V(s0) = 0.000   V(s1) = 0.000   V(s2) = 0.000
iter   1  |   diff:0.64500   |   V(s0) = 0.000   V(s1) = 3.500   V(s2) = 0.000
iter   2  |   diff:0.58050   |   V(s0) = 0.000   V(s1) = 3.815   V(s2) = 0.645
iter   3  |   diff:0.43582   |   V(s0) = 0.581   V(s1) = 3.959   V(s2) = 0.962
iter   4  |   diff:0.30634   |   V(s0) = 0.866   V(s1) = 4.395   V(s2) = 1.272
iter   5  |   diff:0.27571   |   V(s0) = 1.145   V(s1) = 4.670   V(s2) = 1.579
iter   6  |   diff:0.24347   |   V(s0) = 1.421   V(s1) = 4.926   V(s2) = 1.838
iter   7  |   diff:0.21419   |   V(s0) = 1.655   V(s1) = 5.169   V(s2) = 2.075
iter   8  |   diff:0.19277   |   V(s0) = 1.868   V(s1) = 5.381   V(s2) = 2.290
iter   9  |   diff:0.17327   |   V(s0) = 2.061   V(s1) = 5.573   V(s2) = 2.481
iter  10  |   diff:0.15569   |   V(s0) = 2.233   V(s1) = 5.746   V(s2) = 2.654
iter  11  |   diff:0.14012   |   V(s0) = 2.389   V(s1) = 5.902   V(s2) = 2.810
iter  12  |   diff:0.12610   |   V(s0) = 2.529   V(s

In [ ]:
print("Final state values:", state_values)

Final state values: {'s0': 3.7810348735476405, 's1': 7.294006423867229, 's2': 4.202140275227048}


In [ ]:
assert abs(state_values['s0'] - 3.781) < 0.01
assert abs(state_values['s1'] - 7.294) < 0.01
assert abs(state_values['s2'] - 4.202) < 0.01

In [ ]:
def get_optimal_action(mdp, state_values, state, gamma=0.9):
    """ Finds optimal action using formula above. """
    if mdp.is_terminal(state):
        return None

    best_action_index = np.argmax([get_action_value(mdp, state_values, state, a, gamma) for a in mdp.get_possible_actions(state)])
    return mdp.get_possible_actions(state)[best_action_index]

In [ ]:
assert get_optimal_action(mdp, state_values, 's0', gamma) == 'a1'
assert get_optimal_action(mdp, state_values, 's1', gamma) == 'a0'
assert get_optimal_action(mdp, state_values, 's2', gamma) == 'a1'

assert get_optimal_action(mdp, {'s0': -1e10, 's1': 0, 's2': -2e10}, 's0', 0.9) == 'a0', \
    "Please ensure that you handle negative Q-values of arbitrary magnitude correctly"
assert get_optimal_action(mdp, {'s0': -2e10, 's1': 0, 's2': -1e10}, 's0', 0.9) == 'a1', \
    "Please ensure that you handle negative Q-values of arbitrary magnitude correctly"

### Measure agent's average reward
Use append(element) to append to a list

In [ ]:
s = mdp.reset()
rewards = []
for _ in range(10000):
  s,r,done,_ = mdp.step(get_optimal_action(mdp, state_values, s, gamma))
  rewards.append(r)
print("average reward: ", np.mean(rewards))
assert(0.40 < np.mean(rewards) < 0.55)

average reward:  0.4473


##Frozen lake

In [ ]:
from mdp import FrozenLakeEnv

In [ ]:
mdp = FrozenLakeEnv(slip_chance=0)
mdp.render()

*FFF
FHFH
FFFH
HFFG



In [ ]:
def value_iteration(mdp, state_values=None, gamma=0.9, num_iter=1000, min_difference=1e-5):
    """ performs num_iter value iteration steps starting from state_values. Same as before but in a function """
    state_values = state_values or {s: 0 for s in mdp.get_all_states()}
    for i in range(num_iter):

        # Compute new state values using the functions you defined above. It must be a dict {state : new_V(state)}
        new_state_values = {s:get_new_state_value(mdp, state_values, s, gamma) for s in mdp.get_all_states()}

        # Compute difference
        diff = max(abs(new_state_values[s] - state_values[s])
                   for s in mdp.get_all_states())

        print("iter %4i   |   diff: %6.5f   |   V(start): %.3f " %
              (i, diff, new_state_values[mdp._initial_state]))

        state_values = new_state_values
        if diff < min_difference:
            break

    return state_values

In [ ]:
state_values = value_iteration(mdp)

iter    0   |   diff: 1.00000   |   V(start): 0.000 
iter    1   |   diff: 0.90000   |   V(start): 0.000 
iter    2   |   diff: 0.81000   |   V(start): 0.000 
iter    3   |   diff: 0.72900   |   V(start): 0.000 
iter    4   |   diff: 0.65610   |   V(start): 0.000 
iter    5   |   diff: 0.59049   |   V(start): 0.590 
iter    6   |   diff: 0.00000   |   V(start): 0.590 


In [ ]:
s = mdp.reset()
mdp.render()
for t in range(100):
    a = get_optimal_action(mdp, state_values, s, gamma)
    print(a, end='\n\n')
    s, r, done, _ = mdp.step(a)
    mdp.render()
    if done:
        break

*FFF
FHFH
FFFH
HFFG

down

SFFF
*HFH
FFFH
HFFG

down

SFFF
FHFH
*FFH
HFFG

right

SFFF
FHFH
F*FH
HFFG

down

SFFF
FHFH
FFFH
H*FG

right

SFFF
FHFH
FFFH
HF*G

right

SFFF
FHFH
FFFH
HFF*

